In [4]:
import numpy as np
import cv2 
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import sys, os
import pytesseract
from scipy.signal import argrelextrema
import psycopg2
import io
import dbparameter
from basic_functions import Basic
from extractchar_functions import Extract
from getfiles_functions import Getfiles
from postprocess_functions import Postprocess
from preprocess_functions import Preprocess
from subsidiary_functions import Subsidiary
from collections import OrderedDict

extract=Extract()
getfiles=Getfiles()
basic=Basic()
post=Postprocess()
pre=Preprocess()
sub=Subsidiary()

In [3]:
information_data={}
passport_data={}

In [5]:
def isthere_i(array):
    print(array)

In [6]:
def choose_char(matched_array):
    result_string=""
    for i in range(len(matched_array)):                                  
        if matched_array[i][3]==0:
            result_string=result_string+"A"
        elif matched_array[i][3]==1:
            result_string=result_string+"B"
        elif matched_array[i][3]==2:
            result_string=result_string+"C"
        elif matched_array[i][3]==3:
            result_string=result_string+"D"
        elif matched_array[i][3]==4:
            result_string=result_string+"E"    
        elif matched_array[i][3]==5:
            result_string=result_string+"F"
        elif matched_array[i][3]==6:
            result_string=result_string+"G"
        elif matched_array[i][3]==7:
            result_string=result_string+"H"
        elif matched_array[i][3]==8:
            result_string=result_string+"I"
        elif matched_array[i][3]==9:
            result_string=result_string+"J" 
        elif matched_array[i][3]==10:
            result_string=result_string+"K" 
        elif matched_array[i][3]==11:
            result_string=result_string+"L"
        elif matched_array[i][3]==12:
            result_string=result_string+"N"  
        elif matched_array[i][3]==13:
            result_string=result_string+"M" 
        elif matched_array[i][3]==14:
            result_string=result_string+"O" 
        elif matched_array[i][3]==15:
            result_string=result_string+"P" 
        elif matched_array[i][3]==16:
            result_string=result_string+"Q"
        elif matched_array[i][3]==17:
            result_string=result_string+"R"
        elif matched_array[i][3]==18:
            result_string=result_string+"S"
        elif matched_array[i][3]==19:
            check=post.check_I_T(matched_array[i][0])
            if check:
                result_string=result_string+"T"
            else:
                result_string=result_string+"I"
                    
        elif matched_array[i][3]==20:
            result_string=result_string+"U"
        elif matched_array[i][3]==21:
            result_string=result_string+"V"
        elif matched_array[i][3]==22:
            result_string=result_string+"W"
        elif matched_array[i][3]==23:
            result_string=result_string+"X"
        elif matched_array[i][3]==24:
            result_string=result_string+"Y"
        elif matched_array[i][3]==25:
            result_string=result_string+"Z"  

    return result_string        
    

In [7]:
def choose_number(matched_array):
    result_string=""
    for i in range(len(matched_array)):                                  
        if matched_array[i][3]==0:
            result_string=result_string+"0"
        elif matched_array[i][3]==1:
            result_string=result_string+"1"
        elif matched_array[i][3]==2:
            result_string=result_string+"2"
        elif matched_array[i][3]==3:
            result_string=result_string+"3"
        elif matched_array[i][3]==4:
            result_string=result_string+"4"    
        elif matched_array[i][3]==5:
            result_string=result_string+"5"
        elif matched_array[i][3]==6:
           result_string=result_string+"6"
        elif matched_array[i][3]==7:
            result_string=result_string+"7"
        elif matched_array[i][3]==8:
            result_string=result_string+"8"
        elif matched_array[i][3]==9:
            result_string=result_string+"9" 
        elif matched_array[i][3]==10:
            result_string=result_string+"<"
    return result_string

prepared template dataset

In [8]:
def prepare_templates():
    #gereftan template charecter ha b soorat kham
    raw_char_template=getfiles.get_char_templates()
    #tele kardan on raw template ha va greyo reverese kardaneshon
    tele_templates=[]
    for i in range(len(raw_char_template)):
        tele_templates.append(basic.crop_templates(raw_char_template[i],0))
    #resize kardan template ha va albte emal threshhold 0.5
    resized_templates=[]
    for i in range(len(tele_templates)):
        hieght,width=tele_templates[i].shape
        target_width=width*(100/hieght)
        temp=pre.binary_threshold(pre.resize(tele_templates[i],100,int(target_width)),0.5)
        resized_templates.append(temp)
    #padd krrdan template ha bade resize shodan ba 10 ta pixel va normal kardaneshon zira chon bade padd krdn mishan unit 8 va 1 ha mishan 255
    padded_resize_template=[]
    for i in range(len(resized_templates)):
        h,w=resized_templates[i].shape
        #resized_templates[i]=resized_templates[i].astype(np.float32)
        temp=pre.normalize(pre.padding(resized_templates[i],h+10,w+10))
        padded_resize_template.append(temp)
    ############################################################################
    #gereftan template numberha ha b soorat kham
    raw_number_templates=getfiles.get_numbers_templates()
    #tele kardan on raw template ha va greyo reverese kardaneshon
    tele_templates=[]
    for i in range(len(raw_number_templates)):
        tele_templates.append(basic.crop_templates(raw_number_templates[i],0))
    #resize kardan template ha va albte emal threshhold 0.5
    resized_templates=[]
    for i in range(len(tele_templates)):
        hieght,width=tele_templates[i].shape
        target_width=width*(100/hieght)
        temp=pre.binary_threshold(pre.resize(tele_templates[i],100,int(target_width)),0.5)
        resized_templates.append(temp)
    #padd krrdan template ha bade resize shodan ba 10 ta pixel va normal kardaneshon zira chon bade padd krdn mishan unit 8 va 1 ha mishan 255
    padded_resize_number_template=[]
    for i in range(len(resized_templates)):
        h,w=resized_templates[i].shape
        #resized_templates[i]=resized_templates[i].astype(np.float32)
        temp=pre.normalize(pre.padding(resized_templates[i],h+10,w+10))
        padded_resize_number_template.append(temp)

    ############################################################################
    #gereftan template numberha ha b soorat kham
    raw_sexs_templates=getfiles.get_sexs_templates()
    #tele kardan on raw template ha va greyo reverese kardaneshon
    tele_templates=[]
    for i in range(len(raw_sexs_templates)):
        tele_templates.append(basic.crop_templates(raw_sexs_templates[i],0))
    #resize kardan template ha va albte emal threshhold 0.5
    resized_templates=[]
    for i in range(len(tele_templates)):
        hieght,width=tele_templates[i].shape
        target_width=width*(100/hieght)
        temp=pre.binary_threshold(pre.resize(tele_templates[i],100,int(target_width)),0.5)
        resized_templates.append(temp)
    #padd krrdan template ha bade resize shodan ba 10 ta pixel va normal kardaneshon zira chon bade padd krdn mishan unit 8 va 1 ha mishan 255
    padded_resize_sexs_template=[]
    for i in range(len(resized_templates)):
        h,w=resized_templates[i].shape
        #resized_templates[i]=resized_templates[i].astype(np.float32)
        temp=pre.normalize(pre.padding(resized_templates[i],h+10,w+10))
        padded_resize_sexs_template.append(temp)    

    return padded_resize_template,padded_resize_number_template,padded_resize_sexs_template


In [9]:
def mask(image,y_top_left,y_botoom_left,x_top_left,x_top_right):
    for i in range(y_top_left,y_botoom_left):
        for j in range(x_top_left,x_top_right):
            image[i,j]=0

    return image        
    

In [10]:
def find_cut_avg(derivative,cut_index_arr):

    help=0 
    
    for i in range(len(derivative)):
        if(help>60):
            break
        if(derivative[i]==0):
            help=help+1
        if(derivative[i]!=0):
            help=0

        if (derivative[i]==1):
            f_index=i
            for j in range(i,len(derivative)):
                if (derivative[j])==-1:
                    e_index=j
                    cut_index_arr.append(f_index)
                    cut_index_arr.append(e_index+2)
                    break

    for i in range(len(cut_index_arr)):
        if(i==len(cut_index_arr)-1):
            break
        if(i%2!=0):
            start=cut_index_arr[i]
            end=cut_index_arr[i+1]
            avrage=(start+end)/2
            avrage=np.round(avrage,decimals=0)
            avrage=int(avrage)
            cut_index_arr[i]=avrage
            cut_index_arr[i+1]=avrage+1 

In [11]:
def check_extract_char(char_image):
    
    char_height,char_width=char_image.shape
    separated_char=[]

    if char_width<30:
        separated_char.append(char_image)
        return True,separated_char
    else:
        nima=char_image.copy()

        nima=nima*255
        nima=nima.astype(np.uint8)
        cv2.imwrite("nima.jpg",nima)
        cv2.imshow("dss",char_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        copy=char_image.copy()
        filled=sub.fill_hole(copy)
        
        # filled=sub.fill_hole_vertical(filled)
        # cv2.imshow("dss",filled)
        # cv2.waitKey(0)
        # cv2.destroyAllWindows()
        ver_sum=sub.vertical_summation(filled)
        # x= np.arange(len(ver_sum))
        # plt.plot(x, ver_sum)
        # plt.xlabel('Index')
        # plt.ylabel('Value')
        # plt.title('plot')
        # plt.show()
        max=ver_sum.max()
        threshhold=(max/2)-3 
        ver_sum_th=pre.binary_thresh_array(ver_sum,threshhold)
        derative=sub.deriv(ver_sum_th)
        cut_index=[]
        find_cut_avg(derative,cut_index)
        for i in range(0,len(cut_index)):
            if(i%2==0):
                char = char_image[1:char_height,cut_index[i]:cut_index[i+1]]
                separated_char.append(char)
    
        return False,separated_char


In [12]:
def find_cut(derivative,cut_index_arr):

    help=0 
    
    for i in range(len(derivative)):
        if(help>60):
            break
        if(derivative[i]==0):
            help=help+1
        if(derivative[i]!=0):
            help=0

        if (derivative[i]==1):
            f_index=i
            for j in range(i,len(derivative)):
                if (derivative[j])==-1:
                    e_index=j
                    cut_index_arr.append(f_index)
                    cut_index_arr.append(e_index+2)
                    break

In [13]:

def extract_charecter(rectangled_image):

    rec_height,rec_width=rectangled_image.shape
    
    vertical_sum=sub.vertical_summation(rectangled_image)

    vertical_sum_thresh=pre.binary_thresh_array(vertical_sum,1)
    
    cut_index_array=[]
    der=sub.deriv(vertical_sum_thresh)
    find_cut(der,cut_index_array)
    image_of_character=[]
    for i in range(0,len(cut_index_array)):
        if(i%2==0):
            char = rectangled_image[1:rec_height,cut_index_array[i]:cut_index_array[i+1]]
            should_seprate,seprated_char=check_extract_char(char)
            if should_seprate:
                image_of_character.append(seprated_char[0])
            else:
                for j in range(len(seprated_char)):
                    image_of_character.append(seprated_char[j])
                    
    return image_of_character,vertical_sum_thresh,cut_index_array,der

Extract surname , name ,father's name

In [14]:
def ocr_snf(passport):
    #gereftan template haie amade shode
    char_templates,_,_=prepare_templates()
    #bordan dar fazaie greyscale
    orgiganl_image=cv2.cvtColor(passport, cv2.COLOR_BGR2GRAY)
    #check quality if it is lower than 30000 reject it
    try:
        pre.check_iamge_quality(orgiganl_image,20000)
    except ValueError as e:
        sys.exit(str(e))
    #barabar akrdan DPI ha
    orgiganl_image=pre.equalize_dpi(orgiganl_image)
    #gereftan copy az ax asli
    copy_image=orgiganl_image.copy()
    #reverse krdn ax asli
    reversed_img=pre.revrese(copy_image)
    #thereshhold ax asli
    thresholded_img=pre.thresholding(reversed_img,120)
    #normal krdn ax asli
    normaled_img=pre.normalize(thresholded_img)
    cv2.imshow("ssd",normaled_img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    image=normaled_img*255
    image = image.astype(np.uint8)
    #gereftan window ha va rikhtan toie araye
    windows_dict=getfiles.read_images_from_folder("windows")
    #taghir tartib chon mikhahim avaln bre donbal father va bad maskesh kone va badesh surname va akhr name
    new_key_order = ["father.PNG", "surname.PNG", "name.PNG"]
    windows_dict = OrderedDict((key, windows_dict[key]) for key in new_key_order)
    result_string=""
    for key in windows_dict:
        window=windows_dict[key]
        window=pre.revrese(window)
        window=pre.thresholding(window,120)
        window=pre.normalize(window)
        window=window*255
        window = window.astype(np.uint8)
        ################################
        #gerftan jolo fildi k mikhaim
        pt1,pt2,location=basic.templatematching(image,window)
        #mask krdn ghesmat window match shode
        image=mask(image,location[1],location[1]+window.shape[0],location[0],location[0]+window.shape[1])
        #extract joloie on fild
        rectangled_image = normaled_img[pt1[1]:pt2[1], pt1[0]:pt2[0]]
        cv2.imshow("ssd",rectangled_image)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
        #remove kardan noise rectangled
        rectangled_image=rectangled_image*255
        rectangled_image=rectangled_image.astype(np.uint8)
        filtered_image = pre.remove_noise(rectangled_image,20)
        filtered_image=pre.normalize(filtered_image)
        #joda krdn charectare rectangled image
        chareters_image,_,_,_=extract_charecter(filtered_image)  
        #tele kardan template haie charecterha
        tele_char=[]
        for i in range(len(chareters_image)):
            tele_char.append(basic.crop_templates(chareters_image[i],1))
        #door rikhtan on haii az telechar ha ke masahateshon paiin tr az 50 bashe
        index_should_remove=[]
        for i in range(len(tele_char)):
            bool=sub.check_size_of_area(tele_char[i],30)
            if bool:
                pass
            else:
                index_should_remove.append(i)
        index_should_remove=np.sort(index_should_remove)  ##braie inke vaghty pop mikonim az bala pop konim k b moshkel nakhore
        index_should_remove =index_should_remove[::-1]    
        for index in index_should_remove:
            tele_char.pop(index)           
        #resized kardan charha bade tele kardan va hminotr emal threshold
        resized_char=[]
        for i in range(len(tele_char)):
            hieght,width=tele_char[i].shape
            target_width=width*(100/hieght)
            temp=pre.binary_threshold(pre.resize(tele_char[i],100,int(target_width)),0.5)
            resized_char.append(temp)
        #padd krdn char ha bade resize kardan b size hadeaksar templateha b alave 20 va normalsazi doabre chon mibre roe 255, 1 haro
        padded_resize_char=[]
        for i in range(len(resized_char)):
            temp=pre.normalize(pre.padding(resized_char[i],110+20,139+20))##110 ba 139 size maximum az template haie charecterha hastan
            padded_resize_char.append(temp)  
         
        array_of_matched,help=basic.find_char(padded_resize_char,char_templates)

        temp_string=choose_char(array_of_matched)
          
        head, sep, tail = key.partition('.')
        result_string=result_string+head+": "+temp_string+"\n"
        information_data[head]=[temp_string]

    return(result_string)

extract lines(means extract type ,passport number , sex , date of birth)

In [15]:
def ocr_tpsd(passport):
    #gereftan template haie amade shode
    char_templates,number_templates,sexs_templates=prepare_templates()
    #bordan too fazaie graysacle
    orgiganl_image=cv2.cvtColor(passport, cv2.COLOR_BGR2GRAY)
    #barabar kardan dpi ha
    orgiganl_image=pre.equalize_dpi(orgiganl_image)
    #gereftan copy az ax asli
    copy_image=orgiganl_image.copy()
    #reverse krdn ax asli
    reversed_img=pre.revrese(copy_image)
    #thereshhold ax asli
    thresholded_img=pre.thresholding(reversed_img,120)
    #normal krdn ax asli
    normaled_img=pre.normalize(thresholded_img)
    h_sum=sub.horizontal_summation(normaled_img)
    h_thresh=pre.binary_thresh_array(h_sum,20)
    cut_index=[]
    derative=sub.deriv(h_thresh)
    extract.find_cut_for_lines(derative,cut_index)
    size=len(cut_index)
    cut_index=cut_index[size-4:size]
    hhh,www=normaled_img.shape
    line1=normaled_img[cut_index[0]-3:cut_index[1]+3,0:www]
    line2=normaled_img[cut_index[2]-3:cut_index[3]+3,0:www]
    rectangled_image1=line1*255
    rectangled_image2=line2*255
    rectangled_image1=rectangled_image1.astype(np.uint8)
    rectangled_image2=rectangled_image2.astype(np.uint8)
    filtered_image1 = pre.remove_noise(rectangled_image1,20)
    filtered_image2 = pre.remove_noise(rectangled_image2,20)
    filtered_image1=pre.normalize(filtered_image1)
    filtered_image2=pre.normalize(filtered_image2)


    chareters_image1,_,_,_=extract.extract_charecter(filtered_image1)
    chareters_image2,_,_,_=extract.extract_charecter(filtered_image2)
    

    total_result_string=""
    ################extract type of passport#################
    unknown_type=chareters_image1[0]
    unknown_type=basic.crop_templates(unknown_type,1)
    h_un_type,w_un_type=unknown_type.shape
    unknown_type=pre.binary_threshold(pre.resize(unknown_type,100,int((w_un_type*(100/h_un_type)))),0.5)
    unknown_type=pre.normalize(pre.padding(unknown_type,110+20,139+20))
    arr_unknown_type=[]
    arr_unknown_type.append(unknown_type) ##chon onvar toie tabe find char byd araye bedim vgrna dochar moshkel mishe

    char_template_fortype=[char_templates[3],char_templates[15],char_templates[18]]#chon kolna 3 no darim bara type

    type_matched,help=basic.find_char(arr_unknown_type,char_template_fortype)
    if type_matched[0][3]==1:
        total_result_string="Type:"+" P\n"
        information_data['type']="P"
    elif type_matched[0][3]==0:
        total_result_string="Type:"+" D\n" 
        information_data['type']="D"
    elif type_matched[0][3]==3:
        total_result_string="Type:"+" S\n"
        information_data['type']="S"
          
    ################extract type of passport#################
    

    

    ################extract passport number#################
    unknown_number=chareters_image2[1:9]
    unknown_firstchar=chareters_image2[0]
    
    tele_unknown_number=[]
    for unknown in unknown_number:
        temp=basic.crop_templates(unknown,1)
        tele_unknown_number.append(temp)

    resize_unknown_number=[]
    for unknown in tele_unknown_number:
        h,w=unknown.shape
        temp=pre.binary_threshold(pre.resize(unknown,100,int((w*(100/h)))),0.5)
        resize_unknown_number.append(temp)

    padded_unknown_number=[] 
    for unknown in resize_unknown_number:
        temp=pre.normalize(pre.padding(unknown,110+20,139+20))
        padded_unknown_number.append(temp)
    
    arr_unknown_numbers=padded_unknown_number
    

    unknown_firstchar=basic.crop_templates(unknown_firstchar,1)
    h_un_type,w_un_type=unknown_firstchar.shape
    unknown_firstchar=pre.binary_threshold(pre.resize(unknown_firstchar,100,int((w_un_type*(100/h_un_type)))),0.5)
    unknown_firstchar=pre.normalize(pre.padding(unknown_firstchar,110+20,139+20))
    arr_unknown_firstchar=[]
    arr_unknown_firstchar.append(unknown_firstchar) ##chon onvar toie tabe find char byd araye bedim vgrna dochar moshkel mishe
   
    numbers_matched,help=basic.find_char(arr_unknown_numbers,number_templates)
    firstchar_matched,help=basic.find_char(arr_unknown_firstchar,char_templates)

    passportNo_result=choose_number(numbers_matched)
    firstchar_of_passNo=choose_char(firstchar_matched)

     
    total_result_string=total_result_string+"passportNo: "+firstchar_of_passNo+passportNo_result+"\n"
    information_data['passport_no']=firstchar_of_passNo+passportNo_result
    # print(total_result_string)
    ################extract passport number#################


    ################extract birthday#################
    unknown_birthday=chareters_image2[13:19]
    tele_unknown_birthday=[]
    for unknown in unknown_birthday:
        temp=basic.crop_templates(unknown,1)
        tele_unknown_birthday.append(temp)

    resize_unknown_birthday=[]
    for unknown in tele_unknown_birthday:
        h,w=unknown.shape
        temp=pre.binary_threshold(pre.resize(unknown,100,int((w*(100/h)))),0.5)
        resize_unknown_birthday.append(temp)

    padded_unknown_birthday=[] 
    for unknown in resize_unknown_birthday:
        temp=pre.normalize(pre.padding(unknown,110+20,139+20))
        padded_unknown_birthday.append(temp)
    arr_unknown_birthday=padded_unknown_birthday

    birthday_matched,help=basic.find_char(arr_unknown_birthday,number_templates)

    birthday_result=choose_number(birthday_matched)

    total_result_string=total_result_string+"date of birth: "+birthday_result+"\n"
    information_data['date_of_birth']=birthday_result+"000"

    ################extract birthday#################


    ################extract date of expiry#################
    unknown_expiry=chareters_image2[21:27]
    tele_unknown_expiry=[]
    for unknown in unknown_expiry:
        temp=basic.crop_templates(unknown,1)
        tele_unknown_expiry.append(temp)

    resize_unknown_expiry=[]
    for unknown in tele_unknown_expiry:
        h,w=unknown.shape
        temp=pre.binary_threshold(pre.resize(unknown,100,int((w*(100/h)))),0.5)
        resize_unknown_expiry.append(temp)

    padded_unknown_expiry=[] 
    for unknown in resize_unknown_expiry:
        temp=pre.normalize(pre.padding(unknown,110+20,139+20))
        padded_unknown_expiry.append(temp)
    arr_unknown_expiry=padded_unknown_expiry

    expiry_matched,help=basic.find_char(arr_unknown_expiry,number_templates)

    expiry_result=choose_number(expiry_matched)

    total_result_string=total_result_string+"date of expiry: "+expiry_result+"\n"
    information_data['date_of_expiry']=expiry_result+"000"
    
    ################extract date of expiry#################


    ################extract sex#################
    unknown_sex=chareters_image2[20]
    unknown_sex=basic.crop_templates(unknown_sex,1)
    h_un_sex,w_un_sex=unknown_sex.shape
    unknown_sex=pre.binary_threshold(pre.resize(unknown_sex,100,int((w_un_sex*(100/h_un_sex)))),0.5)
    unknown_sex=pre.normalize(pre.padding(unknown_sex,110+20,139+20))
    arr_unknown_sex=[]
    arr_unknown_sex.append(unknown_sex) ##chon onvar toie tabe find char byd araye bedim vgrna dochar moshkel mishe
    
    
    sex_matched,help=basic.find_char(arr_unknown_sex,sexs_templates)

    sex_result=""
    if sex_matched[0][3]==0:
        sex_result="F"
    else:
        sex_result="M"    

    total_result_string=total_result_string+"sex: "+sex_result+"\n"
    information_data['sex']=sex_result
    
    ################extract sex#################

    return(total_result_string)

In [19]:
def main(passport):

   total_result=""

   height,width,_=passport.shape

   upper_half=passport[0:int(height/2),:]
   
   lower_half=passport[int(height/2)+10:height-30,:]
   
   national_code=basic.scan_barcodes(upper_half)

   snf_string=ocr_snf(lower_half)

   tpsd_string=ocr_tpsd(lower_half)

   total_result=str(national_code)+"\n"+snf_string+tpsd_string

   print(total_result)

In [ ]:
passport=cv2.imread("images/passport1.PNG",cv2.IMREAD_COLOR)

with open("images/passport1.PNG", "rb") as image_file:
    image_data = image_file.read()
passport_data["image"]=image_data    

#main(passport)
print(ocr_snf(passport))

print(ocr_tpsd(passport))

# Establish a database connection
conn = psycopg2.connect(dbparametr.dbparams)

# Create a cursor object to interact with the database
cursor = conn.cursor()

try:
    # Insert data into the passport table
    cursor.execute("INSERT INTO passport (image) VALUES (%s)", (passport_data["image"],))

    # Retrieve the generated ID for the passport record
    cursor.execute("SELECT lastval()")
    passport_id = cursor.fetchone()[0]

    # Insert data into the information table
    cursor.execute("""
        INSERT INTO information (
            name, surname, father, type, passport_no, date_of_birth, date_of_expiry, sex, passport_id
        )
        VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)
    """, (
        information_data["name"],
        information_data["surname"],
        information_data["father"],
        information_data["type"],
        information_data["passport_no"],
        information_data["date_of_birth"],
        information_data["date_of_expiry"],
        information_data["sex"],
        passport_id,
    ))

    # Commit the transaction
    conn.commit()

    print("Data inserted successfully!")

except psycopg2.Error as e:
    print("Error:", e)

finally:
    # Close the cursor and connection
    cursor.close()
    conn.close()
